In [1]:
!pip install strsimpy
from strsimpy.jaro_winkler import JaroWinkler
from strsimpy.normalized_levenshtein import NormalizedLevenshtein

import pandas as pd
import numpy as np
import openpyxl
import json
import re
import ast

import rdflib
from rdflib import Literal, RDF, URIRef, Namespace
from rdflib.namespace import RDFS,RDF,XSD

import dateparser
import dateutil.parser as date_parse
dir = "/Users/jaeyoungkim/Desktop/usc/DSCI - 558/project/sub"

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/anaconda3/envs/jaeyoung/bin/python -m pip install --upgrade pip' command.


## 1.  Player

In [2]:
player_df  = pd.read_csv("player_join.csv", index_col = 0)
player_df.head()

,team_history,player_url_x,name,Born,Placeofbirth,Nationality_x,Height_x,Weight,Position(s),Foot,...,Height_y,Nationality_y,Positions,Strengths,Weaknesses,Style_of_Play,Current Team,DOB,birth_block_y,player_id_y
0,"{'0': {'period': '10/2017 - 06/2019', 'team': ...",https://www.worldfootball.net/player_summary/r...,Roger Johnson,['28.04.1983'],"['Ashford', 'England']",['England'],['191cm'],['78kg'],['CentreBack'],['right'],...,191cm,England,Defender (Centre),['Blocking the ball'],['Passing'],"['Indirect set-piece threat', 'Likes to play l...",NaN,28-04-1983,19830428.0,2000086.0
1,"{'0': {'period': '05/2013 - 06/2013', 'team': ...",https://www.worldfootball.net/player_summary/d...,David Bentley,['27.08.1984'],"['Peterborough', 'England']",['England'],['175cm'],['68kg'],"['RightMidfielder', 'RightWinger']",['right'],...,175cm,England,Defender (Centre),NaN,NaN,NaN,NaN,27-08-1984,19840827.0,2000095.0
2,"{'0': {'period': '07/2011 - 06/2012', 'team': ...",https://www.worldfootball.net/player_summary/m...,Michael Johnson,['24.02.1988'],"['Urmston', 'England']",['England'],['183cm'],['79kg'],['DefensiveMidfielder'],['right'],...,183cm,England,Midfielder,NaN,NaN,NaN,NaN,24-02-1988,19880224.0,2000534.0
3,"{'0': {'period': '', 'team': '', 'position': '...",https://www.worldfootball.net/player_summary/j...,Jake Jervis,['17.09.1991'],"['Wolverhampton', 'England']",['England'],['191cm'],NaN,"['RightWinger', 'CentreForward']",['right'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'0': {'period': '', 'team': '', 'position': '...",https://www.worldfootball.net/player_summary/j...,Jake Bidwell,['21.03.1993'],"['Southport', 'England']",['England'],['183cm'],['80kg'],"['LeftBack', 'LeftMidfielder']",['left'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data preprocessing

In [3]:
count = 0
for idx, row in player_df.iterrows():
    count +=1
    if count == 1000:
        break

In [4]:
row

team_history       {'0': {'period': '11/2018 - 12/2018', 'team': ...
player_url_x       https://www.worldfootball.net/player_summary/k...
name                                                      Karl Henry
Born                                                  ['26.11.1982']
Placeofbirth                            ['Wolverhampton', 'England']
Nationality_x                                            ['England']
Height_x                                                   ['183cm']
Weight                                                      ['72kg']
Position(s)                                    ['CentralMidfielder']
Foot                                                       ['right']
Homepage                                                         NaN
sizeofshoe                                                       NaN
birth_block_x                                             19821126.0
player_id_x                                                1000132.0
match_player_id                   

In [5]:
json.loads(re.sub("'", "\"", player_df["team_history"][1000]))

{'0': {'period': '12/2014 - 06/2016',
  'team': 'Bolton Wanderers',
  'position': 'Forward',
  'team_url': 'https://www.worldfootball.net/teams/bolton-wanderers/'},
 '1': {'period': '09/2012 - 06/2014',
  'team': 'Newcastle United Jets',
  'position': 'Forward',
  'team_url': 'https://www.worldfootball.net/teams/newcastle-united-jets/'},
 '2': {'period': '01/2009 - 06/2012',
  'team': 'Aston Villa',
  'position': 'Forward',
  'team_url': 'https://www.worldfootball.net/teams/aston-villa/'},
 '3': {'period': '07/2006 - 01/2009',
  'team': 'Wigan Athletic',
  'position': 'Forward',
  'team_url': 'https://www.worldfootball.net/teams/wigan-athletic/'},
 '4': {'period': '07/2004 - 06/2006',
  'team': 'Birmingham City',
  'position': 'Forward',
  'team_url': 'https://www.worldfootball.net/teams/birmingham-city/'},
 '5': {'period': '03/2000 - 06/2004',
  'team': 'Liverpool FC',
  'position': 'Forward',
  'team_url': 'https://www.worldfootball.net/teams/liverpool-fc/'},
 '6': {'period': '07/199

In [6]:
## Position(s)
def f(x):
    if type(x) == float:
        return x
    else:
        x = ast.literal_eval(x)
        res = [" ".join([s for s in re.split("([A-Z][^A-Z]*)", i) if s]) for i in x]
        return res
    
player_df["Position(s)"] = player_df["Position(s)"].apply(f)

In [7]:
## Positions
def f(x):
    if type(x) == float:
        return x
    tmp = x.split('(')
    if len(tmp) == 1:
        return tmp
    if len(tmp) == 2:
        res = []
        sub = tmp[1]
        sub = re.sub("\(", "" , sub)
        sub = re.sub("\)", "" , sub)
        sub = re.sub(",,",",",sub)
        sub = sub.split(",")
        res = [i +" "+ tmp[0] for i in sub]
        res = [i.strip() for i in res]
        return res
    
def g(x):
    if type(x) == float:
        return x
    elif x[0] == 'C Defensive Midfielder':
        x[0] = 'Centre Defensive Midfielder'
    return x
    
    
player_df["Positions"] = player_df["Positions"].apply(f).apply(g)

## KG

In [8]:
QB = Namespace('http://purl.org/linked-data/cube#')
MYNS = Namespace('http://dsci558.org/myprojectnamespace#')
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
SCHEMA = Namespace("https://schema.org/")
WIKIDATA = Namespace("https://wikidata.org/")

In [9]:
my_kg = rdflib.Graph()
my_kg.bind('qb',QB)
my_kg.bind('my_ns', MYNS)
my_kg.bind('rdfs',RDFS)
my_kg.bind("schema", SCHEMA)
my_kg.bind('rdfs',RDFS)
my_kg.bind('wiki', WIKIDATA)

In [10]:
## Country Entities
country_list_y = list(player_df["Nationality_y"].dropna().drop_duplicates())
tmp = []
for i in [ast.literal_eval(i) for i in list(player_df["Nationality_x"].dropna())] + country_list_y:
    tmp += i
country_list = set(tmp)

for country in country_list:
    revised_name = re.sub("[^a-zA-Z]", "_", country)
    country_uri = URIRef(MYNS[revised_name])
    my_kg.add((country_uri, RDF.type, SCHEMA.Country))
    my_kg.add((country_uri, RDFS.label, Literal(country)))

In [11]:
city_list = set(player_df["Placeofbirth"].dropna().apply(lambda x : ast.literal_eval(x)[0]))

In [12]:
## City Entities
for city in city_list:
    revised_name = re.sub("[^a-zA-Z]", "_", city)
    city_uri = URIRef(MYNS[revised_name])
    my_kg.add((city_uri, RDF.type, MYNS['city']))
    my_kg.add((city_uri, RDF.type, SCHEMA.Place))
    my_kg.add((city_uri, RDFS.label, Literal(city)))

In [13]:
tmp = []
for i in list(player_df["Positions"].dropna()):
    tmp += i
for i in list(player_df["Position(s)"].dropna()):
    tmp += i
position_list = set(tmp)

In [14]:
## Position Entities
for position in position_list:
    revised_name = re.sub("[^a-zA-Z]", "_", position)
    position_uri = URIRef(MYNS[revised_name])
    my_kg.add((position_uri, RDF.type, MYNS['position']))
    my_kg.add((position_uri, RDFS.label, Literal(position)))

In [15]:
## Team Entities

In [16]:
json_list = []
for js in list(player_df["team_history"].dropna()):
    js = re.sub("L'A", "LA", js)
    js = re.sub("l'A", "lA", js)
    js = re.sub("O'H", "OH", js)
    js = re.sub("e'e", "ee", js)
    js = re.sub("d'A", "dA", js)
    js = re.sub("a'a", "aa", js)
    js = re.sub("L'H", "LH", js)
    js = re.sub("d'Á", "dÁ", js)
    js = re.sub("'s ", "s ", js)
    
    js = re.sub("'", "\"", js)
    json_list.append(json.loads(js))

In [17]:
team_list = []
for js in json_list:
    jss = js.values()
    for i in jss:
        team = i["team"]
        if team != "":
            team_list.append(team)
    

In [18]:
for team in set(team_list):
    team = re.sub("[^a-zA-Z]", "_", team)
    team_uri = URIRef(MYNS[team])
    my_kg.add((team_uri, RDF.type, SCHEMA.SportsTeam))
    my_kg.add((team_uri, RDFS.label, Literal(team)))

In [19]:
json.loads(re.sub("'", "\"", player_df["team_history"].dropna()[9]))

{'0': {'period': '02/2019 - 12/2020',
  'team': 'IL Hødd',
  'position': 'Midfielder',
  'team_url': 'https://www.worldfootball.net/teams/il-hoedd/'},
 '1': {'period': '01/2018 - 02/2019',
  'team': 'Nashville SC',
  'position': 'Midfielder',
  'team_url': 'https://www.worldfootball.net/teams/nashville-sc/'},
 '2': {'period': '03/2017 - 12/2017',
  'team': 'Viking FK',
  'position': 'Midfielder',
  'team_url': 'https://www.worldfootball.net/teams/viking-fk/'},
 '3': {'period': '08/2015 - 03/2017',
  'team': 'IL Hødd',
  'position': 'Midfielder',
  'team_url': 'https://www.worldfootball.net/teams/il-hoedd/'},
 '4': {'period': '01/2015 - 08/2015',
  'team': 'Sogndal IL',
  'position': 'Midfielder',
  'team_url': 'https://www.worldfootball.net/teams/sogndal-il/'},
 '5': {'period': '03/2014 - 12/2014',
  'team': 'IL Hødd',
  'position': 'Midfielder',
  'team_url': 'https://www.worldfootball.net/teams/il-hoedd/'},
 '6': {'period': '07/2011 - 03/2014',
  'team': 'Stevenage FC',
  'position':

In [20]:
team_list

['Bromley FC',
 'Charlton Athletic',
 'FC Pune City',
 'Charlton Athletic',
 'Wolverhampton Wanderers',
 'West Ham United',
 'Sheffield Wednesday',
 'Wolverhampton Wanderers',
 'Birmingham City',
 'Cardiff City',
 'Wycombe Wanderers',
 'Tottenham Hotspur',
 'Blackburn Rovers',
 'Tottenham Hotspur',
 'FK Rostov',
 'Tottenham Hotspur',
 'West Ham United',
 'Tottenham Hotspur',
 'Birmingham City',
 'Tottenham Hotspur',
 'Blackburn Rovers',
 'Arsenal FC',
 'Norwich City',
 'Arsenal FC',
 'Leicester City',
 'Manchester City',
 'Luton Town',
 'Salford City',
 'Luton Town',
 'AFC Wimbledon',
 'Luton Town',
 'Plymouth Argyle',
 'Ross County FC',
 'Portsmouth FC',
 'Elazığspor',
 'Birmingham City',
 'Portsmouth FC',
 'Tranmere Rovers',
 'Birmingham City',
 'Carlisle United',
 'Birmingham City',
 'Preston North End',
 'Birmingham City',
 'Swindon Town',
 'Birmingham City',
 'Hereford United',
 'Notts County',
 'Birmingham City',
 'Hereford United',
 'Birmingham City',
 'Queens Park Rangers',
 'B

## Predicate

In [21]:
predicate_uri = MYNS["position"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("position", lang = 'en')))

In [22]:
predicate_uri = MYNS["Foot"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("Foot", lang = 'en')))

In [23]:
predicate_uri = MYNS["shirtNumber"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("shirtNumber", lang = 'en')))

In [24]:
predicate_uri = MYNS["strength"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("strength", lang = 'en')))

In [25]:
predicate_uri = MYNS["weakness"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("weakness", lang = 'en')))

In [26]:
predicate_uri = MYNS["styleofPlay"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("styleofPlay", lang = 'en')))

In [27]:
predicate_uri = MYNS["worldfootball_url"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("worldfootball_url", lang = 'en')))

In [28]:
predicate_uri = MYNS["whoscored_url"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("whoscored_url", lang = 'en')))

In [29]:
predicate_uri = MYNS["Age"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("Age", lang = 'en')))

In [30]:
SCHEMA.name
SCHEMA.brithPlace
SCHEMA.birthDate
SCHEMA.nationality
SCHEMA.height    SCHEMA.QuantitativeValue
SCHEMA.weight    SCHEMA.QuantitativeValue
MYNS["position"]
MYNS["footSize"] SCHEMA.QuantitativeValue
MYNS["footSize"] a SCHEMA.size
MYNS["shirtNumber"] 
MYNS["strength"]
MYNS["weakness"]
MYNS["styleofPlay"]
SCHEMA.startDate SCHEMA.Date
SCHEMA.endDate   SCHEMA.Date
SCHEMA.member   SCHEMA.SportsTeam


SyntaxError: invalid syntax (3736445064.py, line 5)

In [31]:
SCHEMA.brithPlace

rdflib.term.URIRef('https://schema.org/brithPlace')

## DATA

In [32]:
player_df["Foot"].dropna()

0       ['right']
1       ['right']
2       ['right']
3       ['right']
4        ['left']
          ...    
2829    ['right']
2830    ['right']
2831    ['right']
2832    ['right']
2833    ['right']
Name: Foot, Length: 2576, dtype: object

In [33]:
def nulltoNone(x):
    if type(x) == float:
        return "None"
    else:
        return x

In [34]:
for idx, row in player_df.iterrows():     
    player_uri = URIRef(MYNS[nulltoNone(row.player_url_x) + nulltoNone(row.player_url_y)])
    my_kg.add((player_uri, RDF.type, SCHEMA.athlete))
    
    ## url
    if type(row.player_url_x) != float:
        my_kg.add((player_uri, MYNS["worldfootball_url"], Literal(row.player_url_x)))
    if type(row.player_url_y) != float:
        my_kg.add((player_uri, MYNS["whoscored_url"], Literal(row.player_url_y)))
    
    ## name
    if type(row.name) != float:
        my_kg.add((player_uri, SCHEMA.name, Literal(row.name)))
    else:
        my_kg.add((player_uri, SCHEMA.name, Literal(row.Name)))
    
    ## birth date
    try:
        tmp = str(int(row.birth_block_x))
        date = date_parse.parse(tmp[:4] + "-" + tmp[4:6] + "-" + tmp[6:])
        my_kg.add((player_uri, SCHEMA.birthDate, Literal(date,datatype=XSD.date)))
    except:
        try:
            tmp = str(int(row.birth_block_y))
            date = date_parse.parse(tmp[:4] + "-" + tmp[4:6] + "-" + tmp[6:])
            my_kg.add((player_uri, SCHEMA.birthDate, Literal(date,datatype=XSD.date)))
        except:
            a = 1
            
            
    ## place of birth
    if type(row.Placeofbirth) != float:
        tmp = ast.literal_eval(row.Placeofbirth)
        for place in tmp:
            my_kg.add((player_uri, SCHEMA.birthPlace, MYNS[re.sub("[^a-zA-Z]", "_", place)]))
            
            
    ## nationality
    if type(row.Nationality_x) != float:
        tmp = ast.literal_eval(row.Nationality_x)
        my_kg.add((player_uri, SCHEMA.nationality, MYNS[re.sub("[^a-zA-Z]", "_", tmp[0])]))
    elif type(row.Nationality_y) != float:
        my_kg.add((player_uri, SCHEMA.nationality, MYNS[re.sub("[^a-zA-Z]", "_", row.Nationality_y)]))
    
    ## Height
    if type(row.Height_x) != float:
        tmp = row.Height_x.split("'")[1][:3]
        my_kg.add((player_uri, SCHEMA.height,Literal(tmp,datatype=XSD.float) ))
    elif type(row.Height_y) != float:
        tmp = row.Height_y[:3]
        my_kg.add((player_uri, SCHEMA.height,Literal(tmp,datatype=XSD.float) ))
        
    ## Weight
    if type(row.Weight) != float:
        tmp = row.Weight.split("'")[1].split("kg")[0]
        my_kg.add((player_uri, SCHEMA.height,Literal(tmp,datatype=XSD.float) ))
        
    ## Position
    if type(row["Position(s)"])!= float:
        for pos in row["Position(s)"]:
            pos = re.sub("[^a-zA-Z]", "_", pos)
            my_kg.add((player_uri, MYNS["position"], MYNS[pos] ))
    elif type(row.Positions)!= float:
        for pos in row.Positions:
            pos = re.sub("[^a-zA-Z]", "_", pos)
            my_kg.add((player_uri, MYNS["position"], MYNS[pos] ))
            
    ## Foot
    if type(row.Foot) != float:
        foot = row.Foot.split("'")[1]
        my_kg.add((player_uri, MYNS["Foot"], Literal(foot)))
        
    ## Age
    try:
        a = int(row.Age)
        my_kg.add((player_uri, MYNS["Age"], Literal(row.Age)))
    except:
        a = 1
        
    ## Current Team
    ## Past team
        
        
    
    

In [35]:
int(row.Age)

21

In [36]:
player_df["Current Team"].isna().sum()

1721

In [37]:
if player_df["Current Team"][2]:
    print(1)  

1


In [38]:
player_df.columns

Index(['team_history', 'player_url_x', 'name', 'Born', 'Placeofbirth',
       'Nationality_x', 'Height_x', 'Weight', 'Position(s)', 'Foot',
       'Homepage', 'sizeofshoe', 'birth_block_x', 'player_id_x',
       'match_player_id', 'player_url_y', 'Name', 'Shirt Number', 'Age',
       'Height_y', 'Nationality_y', 'Positions', 'Strengths', 'Weaknesses',
       'Style_of_Play', 'Current Team', 'DOB', 'birth_block_y', 'player_id_y'],
      dtype='object')

In [39]:
player_df["Current Team"]

0                           NaN
1                           NaN
2                           NaN
3                           NaN
4                           NaN
                 ...           
3041                    Arsenal
3042    Wolverhampton Wanderers
3043                     Burton
3044               Hajduk Split
3045                   FC Porto
Name: Current Team, Length: 3046, dtype: object

In [40]:
player_df["Positions"]

0                          [Centre Defender]
1                          [Centre Defender]
2                               [Midfielder]
3                                        NaN
4                                        NaN
                        ...                 
3041                       [Centre Defender]
3042                       [Left Midfielder]
3043                               [Forward]
3044    [Centre Midfielder, Left Midfielder]
3045           [Centre Attacking Midfielder]
Name: Positions, Length: 3046, dtype: object

In [86]:
my_kg.serialize('data.ttl', format="turtle") 